# Get size and coordinates of every german county
Used to map the Covid-19 cases of germany on a map

In [1]:
from json2xml.utils import readfromurl    # to convert the json from the API into a python list
import numpy as np    # casting the polygons into np.arrays allows to transpose them easily with the .T function
import matplotlib.pyplot as plt    # to plot the counties
import json    # to save the data as json in a file
import os.path    # to check if their's a local file with the API-data or if a new API pull is inevitable

## Control
Set variables to true to trigger the action described by the variables name.

In [2]:
show_examples_for_functions_for_easier_use_of_the_data = False
plot_unpolished_data_for_rough_overview = False    # takes approximately 2 minutes longer
plot_modified_data_for_rough_overview = False    # takes approximately 2 minutes longer
new_pull_from_api = False
only_plot_counties_with_an_index_higher_than_this = 413    # highest: 412

## Get the data and safe it in a file
Set the correct URL to the arcgis services. From there we get multiple geometrical informations about the german counties, for us relevant are their size, their length, their shape and their inhabitants number.
<br/>
In the second step the json from the URL gets interpreted as a dictionary.

In [3]:
# if the data does not contain that many counties, the program raises an error and pulls from a local backup of the API pull
number_of_counties = 412

In [4]:
# check if a local pull of the API exists and otherwise initiate a new pull from the API
if not(os.path.isfile('german_county_data.txt')): new_pull_from_api = True

# check if new pull from the API is necessary or wished and if it is even possible otherwise "pull" from local backup
if new_pull_from_api:
    # get data
    url = "https://services7.arcgis.com/mOBPykOjAyBO2ZKk/arcgis/rest/services/RKI_Landkreisdaten/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json"
    data = readfromurl(url)
    # data is maybe faulty: report to user and use local backup of old API pull
    if len(data["features"]) != number_of_counties:
        new_pull_from_api = False
        print("The provided data from the API does not have the preset number of counties of {}, it has {}.".format(number_of_counties, len(data["features"])))
    # data seems to be fine: continue
    else:    # only store flawless data
        with open("german_county_data.txt", "w") as file:
            file.write(json.dumps(data))
        print("Data directly from API is ready to go!")

# Use data from local backup originating from old API pull
# new_pull_from_api gets modified in the if-statement - therefore no else-statement here
if not(new_pull_from_api):
    with open("german_county_data.txt", "r") as file:
        data = json.loads(file.read())
    print("Data from old API-pull is ready to go!")

Data from old API-pull is ready to go!


## Draw the counties
Every countys shape can be reached by calling the value of "geometry" and afterwards "rings". Kept in a very basic way to demonstrate the used data.

In [5]:
if plot_unpolished_data_for_rough_overview:
    plt.figure(figsize=(20, 20))    # enlarge plot
    # put all shapes into the plot
    for county in data['features']:
        for polygon in county['geometry']['rings']:
            x,y = np.array(polygon).T
            plt.fill(x, y)
    plt.show()    # draw plot

## Functions for easier use of the data
### county_name_and_id(county): returns string
returns a string containing the ID of a county and it's name separated by a whitespace.
### counties_name_and_id(counties): returns string
calls county_name_and_id() for every county in counties (separated by a ", " and the last with an "and")
### draw_polygon(polygon, plot_immediately, color): returns void (could show a plot)
**polygon**<br/>
-> shape of the polygon/county
<br/>
**plot_immediately** (default: True)<br/>
-> Determines whether "plt.show()" gets executed or not. If not the plot will be drawn with all other plots created in the same manner whenever plt.show() is called - allows to draw multiple polygons in one coordinate system
<br/>
**color** (default: None (standard color sequence))<br/>
-> sets the color of the polygon
### draw_polygons(polygons, plot_single, plot_immediately , plt_color):
**polygons**<br/>
-> shapes of polygons/counties
<br/>
**plot_single** (default: False)<br/>
-> determines wether every polygon shall be drawn in it's on coordinate system or if all polygons shall be accumulated in one
<br/>
**plot_immediately** (default: True)<br/>
-> Determines whether "plt.show()" gets executed in the end or not. If not the plot will be drawn with all other plots created in the same manner whenever plt.show() is called - allows to draw multiple polygons in one coordinate system
<br/>
**color** (default: None (standard color sequence))<br/>
-> sets the color of the polygons

In [6]:
def county_name_and_id(county):
    return '{} {}'.format(county['attributes']['OBJECTID'],county['attributes']['county'])

def counties_name_and_id(counties):
    # first county, no comma:
    result = county_name_and_id(counties[0])
    # all following with comma
    for county in counties[1:-1]:
        result += ', {}'.format(county_name_and_id(county))
    # last one with " and "
    result += ' and {}'.format(county_name_and_id(counties[-1]))
    return result

if show_examples_for_functions_for_easier_use_of_the_data:
    print(county_name_and_id(data['features'][0]))
    # e.g. to display county 43 to 69
    print(counties_name_and_id(data['features'][42:69]))
    print()
    # e.g. to display all counties with their OBJECTID
    for county in data['features']:
        print(county_name_and_id(county))

In [7]:
def draw_polygon(polygon, plot_immediately = True, plt_color = None):
    x,y = np.array(polygon).T
    if plt_color:
        plt.fill(x, y, color = plt_color)
    else:
        plt.fill(x, y)
    if plot_immediately: plt.show()


def draw_polygons(polygons, plot_single = False, plot_immediately = True, plt_color = None):
    for polygon in polygons[:-1]:
        draw_polygon(polygon, plot_single, plt_color)
    if len(polygons):
        draw_polygon(polygons[-1], plot_immediately, plt_color)




if show_examples_for_functions_for_easier_use_of_the_data:
    # e.g. to display this rectangle in
    print("all default: color by standard color scheme and plot immediately")
    draw_polygon([[1, 1], [1, 2], [2, 2], [2, 1]])


    print("change color to red ('r')")
    draw_polygon([[1, 1], [1, 2], [2, 2], [2, 1]], True, "r")


    print("change color to red ('r') and don't draw it")
    draw_polygon([[1, 1], [1, 2], [2, 2], [2, 1]], False, "r")
    print("cutout center of the red square from directly above and plot it (color is white)")
    draw_polygon([[1.25, 1.25], [1.25, 1.75], [1.75, 1.75], [1.75, 1.25]], True, "w")

    print("drawing multiple polygons into one plot with the draw_polygons function:")
    draw_polygons([[[-1, 1], [-1, 2], [-2, 2], [-2, 1]],
                   [[1.25, 1.25], [1.25, 1.75], [1.75, 1.75], [1.75, 1.25]]],
                  False, True, "r")

## Backup shapes and filter the counties with multiple polygons
Every countys shape can be reached by calling the value of "geometry" and afterwards "rings".
<br/>
In the following the polygons get a second refernce as backup: "raw_rings"
Because the polygons are by far the biggest part of the data we only want to duplicate them if necessary that's why only a reference and no copy.
This bit risky but more efficient option is also chosen because the original data
is backupped as a whole in "german_county_data.txt" as well.
<br/><br/>
Visualization of the refernce and copy properties:

In [8]:
a = [1, 2, 3]
backup_of_a = a
a[1] = 5    # destroys the backup
print(a, backup_of_a)
a = list()    # sets a new refernce
print(a, backup_of_a)

[1, 5, 3] [1, 5, 3]
[] [1, 5, 3]


Counties with no enclaves or whatsoever have one polygon stored in the value corresponding to "rings",
all others have the cutouts or enclaves stored as extra polygons.
<br/>
So we go through all the counties and if one of them has more than one polygon stored as shape, we store it's index in index_of_counties_with_multiple_polygons
<br/><br/>
**!!!Remember to store the original polygons under data['features'][county_index]['geometry']['raw_rings'] before you modify it!!!**<br/>
**!!!And only modify them by setting the whole array, by manipulating single elements no new reference is created!!!**

In [9]:
index_of_counties_with_multiple_polygons = list()
for county_index in range(len(data['features'])):
    countys_geometry = data['features'][county_index]['geometry']
    # referencing the original polygons
    countys_geometry['raw_rings'] = countys_geometry['rings']
    # store the index of every county with more than one polygon
    if len(countys_geometry['rings']) > 1:
        index_of_counties_with_multiple_polygons.append(county_index)

## Check the counties with multiple polygons
If a county has more than one polygon stored as shape, it must be checked whether it's polygons are cutouts or real area.
The fastest way is to do this by hand, so there are three functions to support that:<br/>
draw_polygons_of_county(county_index, draw_original_polygons = True) (plots polygons of given county),<br/>
set_polygons_of_county(county_index, areas_or_cutouts_list) (processes the decision of the user) and <br/>
generator_index_of_counties_with_multiple_polygons() (a generator to iterate over the counties with multiple polygons)
<br/><br/>
set_polygons_of_county executes the preferred action for the polygon and stores the modified version in 'rings':
- just "add" the polygon to 'rings',
- "remove"/cut the polygon out of it's predecessor in the polygon list
  (this is done by adding it to the prior polygon and adding the last coordinate again,
  so technicaly there's a small slice cut out of the reala area, this is no problem for our purposes)
- or simply "exclude" the polygon.

In [10]:
def draw_polygons_of_county(county_index, draw_original_polygons = True):
    # only_plot_counties_with_an_index_higher_than_this
    # gets set at the begining of the file to execute faster
    if only_plot_counties_with_an_index_higher_than_this <= county_index:
        county = data['features'][county_index]
        # title of the plot
        plt.title("{} with {} polygons".format(county_name_and_id(county), len(county['geometry']['raw_rings'])))
        plt.rcParams["figure.figsize"]= (20, 20)    # enlarge plot
        if draw_original_polygons:
            draw_polygons(county['geometry']['raw_rings'])
        else:
            draw_polygons(county['geometry']['rings'])

In [11]:
def set_polygons_of_county(county_index, areas_or_cutouts_list):
    county = data['features'][county_index]
    # check if every polygon gets a value:
    # - "exclude" to discard polygon
    # - "add" to leave as is (for enclaves)
    # - "remove" to trade as cutout: add it to the prior polygon (not possible for first polygon)
    if len(county['geometry']['raw_rings']) != len(areas_or_cutouts_list):
        raise Exception("lists of areas and cutouts and the polygon list do not have the same length: {} =/= {}".format(
        len(areas_or_cutouts_list), len(county['geometry']['raw_rings'])))

    # create new object at 'rings' to not destroy the backup (explained in the docstring)
    county['geometry']['rings'] = list()

    # process user information:
    for polygon_index in range(len(county['geometry']['raw_rings'])):
        if areas_or_cutouts_list[polygon_index] == "add":
            county['geometry']['rings'].append(county['geometry']['raw_rings'][polygon_index].copy())
        elif areas_or_cutouts_list[polygon_index] == "remove":
            if polygon_index == 0:
                raise Exception("The first polygon can't be cutout of it's prior polygon because there's no prior polygon")
            county['geometry']['rings'][-1] += (county['geometry']['raw_rings'][polygon_index].copy()) + [county['geometry']['rings'][-1][-1]]
        # nothing needs to be done if the command is "exclude", but if the command is unknown an error shall be raised
        elif not(areas_or_cutouts_list[polygon_index] == "exclude"):
            raise Exception("The command {} is not known - use 'add', 'remove' or 'exclude'".format(areas_or_cutouts_list[polygon_index]))

    # draw to control
    draw_polygons_of_county(county_index, False)

In [12]:
def generator_index_of_counties_with_multiple_polygons():
    for index_of_county_with_multiple_polygons in index_of_counties_with_multiple_polygons:
        yield index_of_county_with_multiple_polygons
unchecked_counties = generator_index_of_counties_with_multiple_polygons()

In [13]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "remove"])

In [14]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "add"])

In [15]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "add"])

In [16]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "add", "add", "add", "add", "add", "add", "add", "add", "add", "add", "add", "add", "add", "add", "add"])

In [17]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "add", "add", "add"])

In [18]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "add", "add", "add", "add", "add", "add", "add"])

In [19]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "remove"])

In [20]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "add", "add"])

In [21]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "add", "add", "add"])

In [22]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "add"])

In [23]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "add", "add", "add", "add"])

In [24]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "add"])

In [25]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "add"])

In [26]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "remove"])

In [27]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "add", "add"])

In [28]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "remove", "remove"])

In [29]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "remove"])

In [30]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "add", "add", "add", "add", "add", "add"])

In [31]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "add", "add"])

In [32]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "add", "add", "add", "add"])

In [33]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "remove"])

In [34]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "add"])

In [35]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "add", "add"])

In [36]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "add"])

In [37]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "add", "add"])

In [38]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "add", "add"])

In [39]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "add"])

In [40]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "add", "add", "add", "add"])

In [41]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "remove", "add", "add"])

In [42]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "add"])

In [43]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "add"])

In [44]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "remove"])

In [45]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "add", "remove"])

In [46]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "add", "remove"])

In [47]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "remove"])

In [48]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "add", "add"])

In [49]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "remove", "remove"])

In [50]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "remove"])

In [51]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "add"])

In [52]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "remove"])

In [53]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "add"])

In [54]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "remove"])

In [55]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "add"])

In [56]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "add"])

In [57]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "add", "add", "add", "remove", "remove", "remove"])

In [58]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "add"])

In [59]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "add", "add", "add", "remove", "remove", "remove", "remove", "remove", "remove", "remove"])

In [60]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "add", "add", "remove"])

In [61]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "add", "add", "add"])

In [62]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "add", "add", "add"])

In [63]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "add", "add", "remove"])

In [64]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "remove", "remove"])

In [65]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "add", "add", "add", "remove"])

In [66]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "remove", "remove", "remove"])

In [67]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "remove"])

In [68]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "remove"])

In [69]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "remove"])

In [70]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "remove"])

In [71]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "remove"])

In [72]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "add"])

In [73]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "add"])

In [74]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "add", "remove"])

In [75]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "remove", "remove", "remove"])

In [76]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "remove"])

In [77]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "remove"])

In [78]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "add"])

In [79]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "remove"])

In [80]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "remove"])

In [81]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "remove"])

In [82]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "remove"])

In [83]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "add"])

In [84]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "remove", "remove"])

In [85]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "add", "add", "remove"])

In [86]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "remove"])

In [87]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "remove"])

In [88]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "add"])

In [89]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "remove"])

In [90]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "add"])

In [91]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "remove"])

In [92]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "add"])

In [93]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "add"])

In [94]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "add", "add"])

In [95]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "remove"])

In [96]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "add", "add"])

In [97]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "add", "remove"])

In [98]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "add"])

In [99]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add"]*37)

In [100]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "add", "add", "add", "add", "add", "remove", "remove"])

In [101]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add"]*21)

In [102]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "add", "add"])

In [103]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "add"])

In [104]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "add", "add", "remove"])

In [105]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "add"])

In [106]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "remove", "remove"])

In [107]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "add", "add", "remove"])

In [108]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "add", "remove", "remove"])

In [109]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "add", "remove"])

In [110]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "remove"])

In [111]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "add"])

In [112]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "remove"])

In [113]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "remove"])

In [114]:
current_county_index = next(unchecked_counties)
draw_polygons_of_county(current_county_index)
set_polygons_of_county(current_county_index, ["add", "add", "add", "add", "remove"])

## Save modified data in file

In [115]:
with open("modified_german_county_data.txt", "w") as file:
    file.write(json.dumps(data))

## Draw the counties
Every countys shape can be reached by calling the value of "geometry" and afterwards "rings". Kept in a very basic way to demonstrate the used data.

In [116]:
if plot_modified_data_for_rough_overview:
    plt.figure(figsize = (20, 20))    # enlarge plot
    for county in data['features']:
        for polygon in county['geometry']['rings']:
            x,y = np.array(polygon).T
            plt.fill(x, y)
    plt.show()